In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

data = pd.read_csv('democracy.csv')

In [3]:
modern_data = data[data['Year'] > 2016]
modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)
non_modern_data = data[data['Year'] <= 2016]
non_modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
non_modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)


C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\1065609558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\1065609558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\1065609558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [4]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'World']
for continent in continents:
    if continent in modern_data.columns:
        modern_data.drop(continent, axis=1, inplace=True)

In [5]:
all_countries = modern_data['Entity'].unique()
predictions_country = {'Curve': [], 'Country': []}

for country in all_countries:
    country_data = modern_data[modern_data['Entity'] == country]
    X = country_data['Year'].values.reshape(-1, 1)
    y = country_data['Index'].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)
    slope = model.coef_[0][0]
    intercept = model.intercept_[0]
    curve = [slope, intercept]
    predictions_country['Curve'].append(curve)
    predictions_country['Country'].append(country)

predictions_country = pd.DataFrame(predictions_country)
predictions_country.head()


,Curve,Country
0,"[-0.05557142857142856, 112.50266666666666]",Afghanistan
1,"[-0.0057142857142857195, 12.035000000000009]",Albania
2,"[-0.006228571428571419, 12.868933333333315]",Algeria
3,"[0.00048571428571428214, -0.632733333333326]",Angola
4,"[0.016228571428571422, -31.98359999999999]",Argentina


In [6]:
points = []
country = []
code = []

predictions_code = modern_data[modern_data['Year'] == 2020]

x = np.arange(2023, 2031, 1)
for i in range(len(predictions_country['Curve'])):
    for j in x:
        value = predictions_country['Curve'][i][0] * j + predictions_country['Curve'][i][1]
        points.append(value)
        country.append(predictions_country['Country'][i])
        code.append(predictions_code['Code'].iloc[i])

all_years = []    
for i in range(len(all_countries)):
    for j in x:
        all_years.append(j)
        

predictions = pd.DataFrame({'Entity': country, 'Code': code, 'Year': all_years, 'Index': points})
predictions = predictions.sort_values(by=['Entity', 'Year'])
predictions
        

,Entity,Code,Year,Index
0,Afghanistan,AFG,2023,0.081667
1,Afghanistan,AFG,2024,0.026095
2,Afghanistan,AFG,2025,-0.029476
3,Afghanistan,AFG,2026,-0.085048
4,Afghanistan,AFG,2027,-0.140619
...,...,...,...,...
1435,Zimbabwe,ZWE,2026,0.276581
1436,Zimbabwe,ZWE,2027,0.273952
1437,Zimbabwe,ZWE,2028,0.271324
1438,Zimbabwe,ZWE,2029,0.268695


In [7]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
for continent in continents:
    if continent in predictions['Entity']:
        predictions = predictions.drop(continent)
filtered = predictions[predictions['Entity'] != 'World']

filtered['Index'] = round(filtered['Index'], 3)
filtered

,Entity,Code,Year,Index
0,Afghanistan,AFG,2023,0.082
1,Afghanistan,AFG,2024,0.026
2,Afghanistan,AFG,2025,-0.029
3,Afghanistan,AFG,2026,-0.085
4,Afghanistan,AFG,2027,-0.141
...,...,...,...,...
1435,Zimbabwe,ZWE,2026,0.277
1436,Zimbabwe,ZWE,2027,0.274
1437,Zimbabwe,ZWE,2028,0.271
1438,Zimbabwe,ZWE,2029,0.269


In [47]:
predict = pd.concat([non_modern_data, modern_data, filtered])
predict.sort_values(by=['Entity', 'Year'], inplace=True)
predict = predict[round(predict['Index'], 3) <= 1.00]
predict.to_csv('predictions.csv', index=True)

In [48]:
world = pd.read_csv('world_data.csv')
world_predictions = world[world['Year'] >= 2020]
world_modern = world[world['Year'] >= 1900]
columns = world_modern.columns[3:]

for i in range(len(world_modern)):
    numbers = []
    for column in columns:
        numbers.append(world_modern[column].iloc[i])
    numbers_sum = sum(numbers)
    percentages = numbers / numbers_sum
    world_modern['Authoritarian'].iloc[i] = percentages[0]
    world_modern['Partially Authoritarian'].iloc[i] = percentages[1]
    world_modern['Partial Democracy'].iloc[i] = percentages[2]
    world_modern['Liberal Democracy'].iloc[i] = percentages[3]

world_modern.drop('Unnamed: 0', axis=1, inplace=True)
world_modern

C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\634543747.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_modern['Authoritarian'].iloc[i] = percentages[0]
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\634543747.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_modern['Partially Authoritarian'].iloc[i] = percentages[1]
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\634543747.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,Entity,Year,Authoritarian,Partially Authoritarian,Partial Democracy,Liberal Democracy
111,World,1900,0.785124,0.173554,0.016529,0.024793
112,World,1901,0.776860,0.181818,0.016529,0.024793
113,World,1902,0.780488,0.170732,0.016260,0.032520
114,World,1903,0.785714,0.166667,0.015873,0.031746
115,World,1904,0.773438,0.171875,0.023438,0.031250
...,...,...,...,...,...,...
229,World,2018,0.129213,0.342697,0.308989,0.219101
230,World,2019,0.134831,0.348315,0.297753,0.219101
231,World,2020,0.134831,0.348315,0.314607,0.202247
232,World,2021,0.157303,0.331461,0.325843,0.185393


In [39]:
prediction_years = [2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
type_ = []

for i in prediction_years:
    year_data = filtered[filtered['Year'] == i]
    print('The Length of {} is {}'.format(i, len(year_data)))
    for j in range(len(year_data)):
        if year_data.iloc[j]['Index'] >= 0.8:
            type_.append('Liberal Democracy')
        elif 0.6 <= year_data.iloc[j]['Index'] < 0.8:
            type_.append('Partial Democracy')
        elif 0.4 <= year_data.iloc[j]['Index'] < 0.6:
            type_.append('Partially Authoritarian')
        elif year_data.iloc[j]['Index'] < 0.4:
            type_.append('Authoritarian')
first_data = type_[:180]
second_data = type_[180:360]
third_data = type_[360:540]
fourth_data = type_[540:720]
fifth_data = type_[720:900]
sixth_data = type_[900:1080]
seventh_data = type_[1080:1260]
eight_data = type_[1260:1440]   
    

The Length of 2023 is 180
The Length of 2024 is 180
The Length of 2025 is 180
The Length of 2026 is 180
The Length of 2027 is 180
The Length of 2028 is 180
The Length of 2029 is 180
The Length of 2030 is 180


In [43]:
data = [first_data, second_data, third_data, fourth_data, fifth_data, sixth_data, seventh_data, eight_data]
all_percents = []
for year in data:
    auth = 0
    part_auth = 0
    part_dem = 0
    dem = 0
    for i in year:
        if i == 'Authoritarian':
            auth += 1
        elif i == 'Partially Authoritarian':
            part_auth += 1
        elif i == 'Partial Democracy':
            part_dem += 1
        elif i == 'Liberal Democracy':
            dem += 1
    all = [auth, part_auth, part_dem, dem]
    total = sum(all)
    for i in all:
        percents = i / total
        all_percents.append(percents)
    print(len(all_percents))

4
8
12
16
20
24
28
32


In [59]:
i = 0
world_modern = pd.DataFrame(world_modern)

for j in range(8):
    new_row = {'Entity': 'World', 'Year': prediction_years[j], 'Authoritarian': all_percents[i], 'Partially Authoritarian': all_percents[i+1], 'Partial Democracy': all_percents[i+2], 'Liberal Democracy': all_percents[i+3]}
    i += 4
    world_modern = world_modern.append(new_row, ignore_index=True)
world_modern


C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\625223802.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_modern = world_modern.append(new_row, ignore_index=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\625223802.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_modern = world_modern.append(new_row, ignore_index=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\625223802.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_modern = world_modern.append(new_row, ignore_index=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_19944\625223802.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_m

,Entity,Year,Authoritarian,Partially Authoritarian,Partial Democracy,Liberal Democracy
0,World,1900,0.785124,0.173554,0.016529,0.024793
1,World,1901,0.776860,0.181818,0.016529,0.024793
2,World,1902,0.780488,0.170732,0.016260,0.032520
3,World,1903,0.785714,0.166667,0.015873,0.031746
4,World,1904,0.773438,0.171875,0.023438,0.031250
...,...,...,...,...,...,...
134,World,2026,0.461111,0.172222,0.172222,0.194444
135,World,2027,0.455556,0.166667,0.194444,0.183333
136,World,2028,0.461111,0.161111,0.194444,0.183333
137,World,2029,0.466667,0.150000,0.188889,0.194444


In [60]:
world_modern.to_csv('world_predictions.csv', index=False)